In [1]:
import os
import numpy as np
import pandas as pd
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging
from sklearn.model_selection import train_test_split
import torch
import random
from tqdm import tqdm
from transformers import BertJapaneseTokenizer, BertForMaskedLM
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification,AdamW, BertConfig, BertModel, BertPreTrainedModel
import pytorch_lightning as pl
#MODEL_NAME='bert-base-cased'
from torch import nn
import MeCab

In [9]:
df_Tokyo = pd.read_csv('/home/is/shuntaro-o/dev/persons_move_analysis/data/use_new_model/202271month_per_hour_noGeo_adEstimate_Tokyo_ad2zimethestimate.csv')

In [10]:
df_Tokyo_station = df_Tokyo[df_Tokyo["Tokyo_code"]==533946]
df_Tokyo_station = df_Tokyo_station.reset_index(drop=True)

In [13]:
df_Tokyo_station

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,created_at,retweets,replies,likes,quote_count,author_id,username,author_followers,author_tweets,author_description,author_location,code,Tokyo_code,creatid_at_day,creatid_at_hour
0,46,569,80,RT : 2022-07-01 01:59:23 のおすすめのアイテムは↓ 禿防止のサプリを...,2022-07-01_01:59:58,3.0,0.0,0.0,0.0,1371788620009381892,TryPython,270.0,864448,※このアカウントはBOTになりました。,NaN,5339,533946,01,01
1,161,1657,187,みりほースタンバイ 時報ツイートまで約３秒です 2022/07/01 03:59:5...,2022-07-01_03:59:57,0.0,0.0,0.0,0.0,4580027352,000ms_ho,1681.0,634422,みりほーちゃんがフォロバしたユーザーが、よるほーや時報をピッタリに呟くと数秒後にピタオメリプ...,みりほーのトリセツ：詳しくは、Web見てね♪ 固定ツイートも見てね♪,5339,533946,01,03
2,163,1669,時々女子大生です☆〜（ゝ。∂）,上野動物園キャンパス,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5339,533946,,
3,166,1679,208,こちら03:59。鳥のさえずりが聞こえ始めてます。,2022-07-01_03:59:57,0.0,0.0,2.0,0.0,1416380292512706560,NARIPON_T,192.0,4064,💢VALORANT┊最高A3 現A2🔫APEX┊( ᐙ )🕹ガンエボ┊シルバー2,NaN,5339,533946,01,03
4,220,2361,391,たこ焼きとからあげクンな晩ご飯えびが好き♡ひじきとワカメ入りサラダ。#晩御飯 ＃晩ごはん #...,2022-07-01_04:59:54,3.0,0.0,34.0,0.0,2491409160,7717Hi,265.0,2349,好きなものを自由に。好きなことが一緒の方のフォロー大歓迎です。料理、お酒、セカオワ、vaun...,NaN,5339,533946,01,04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,30969,349991,152,おはようございます。お返事ありがとうございます。縦の三玖ちゃんを求めておりましたので今回は...,2022-07-31_08:59:58,0.0,0.0,1.0,0.0,942390297715023872,ldh_36829,146.0,772,交換垢 成人済み,NaN,5339,533946,31,08
639,31094,351502,203,RT : 秋葉原駅内イベント開催中です！ HOW HOUSE PICK UP CREATOR...,2022-07-31_11:59:58,6.0,0.0,0.0,0.0,4310858292,chii_nekomatagi,338.0,112826,雑食系。面白そうなモノはまず齧ってみる。典型「熱しやすく冷めやすい」ASD系の障害手帳持ち。...,南関東のブラックホール,5339,533946,31,11
640,31144,352108,321,久しぶりに青春18で東京いくのもありか・・・,2022-07-31_12:59:58,0.0,0.0,3.0,0.0,1507245180138651650,0108siHiroto,229.0,6328,京都工芸繊維大 情報工学 B1 | KMC,京都,5339,533946,31,12
641,31219,352828,87,RT : やった！あかりちゃんの絵柄が入ったうちわもらいました。あとブロマイドも当選。しかも...,2022-07-31_14:59:58,1.0,0.0,0.0,0.0,1477765710663516160,yryryrmgr,40.0,6156,#ゆるゆり活動日記 をひたすら巡回するだけの非公式アカウントです。ゆる〜くやってます。,NaN,5339,533946,31,14


In [11]:
df_Tokyo_station["created_at"]=df_Tokyo_station["created_at"].astype(str)
created_time=df_Tokyo_station.created_at.values
creatid_at_day_list = []
creatid_at_hour_list = []
for i in created_time:
    creatid_at_day_list.append(i[8:10])
    creatid_at_hour_list.append(i[11:13])
creatid_at_day_list = pd.Series(creatid_at_day_list)
creatid_at_hour_list = pd.Series(creatid_at_hour_list)
df_Tokyo_station=pd.concat([df_Tokyo_station,creatid_at_day_list],axis=1)
df_Tokyo_station=df_Tokyo_station.rename(columns={0: 'creatid_at_day'})
df_Tokyo_station=pd.concat([df_Tokyo_station,creatid_at_hour_list],axis=1)
df_Tokyo_station=df_Tokyo_station.rename(columns={0: 'creatid_at_hour'})

In [12]:
df_Tokyo_station

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,text,created_at,retweets,replies,likes,quote_count,author_id,username,author_followers,author_tweets,author_description,author_location,code,Tokyo_code,creatid_at_day,creatid_at_hour
0,46,569,80,RT : 2022-07-01 01:59:23 のおすすめのアイテムは↓ 禿防止のサプリを...,2022-07-01_01:59:58,3.0,0.0,0.0,0.0,1371788620009381892,TryPython,270.0,864448,※このアカウントはBOTになりました。,NaN,5339,533946,01,01
1,161,1657,187,みりほースタンバイ 時報ツイートまで約３秒です 2022/07/01 03:59:5...,2022-07-01_03:59:57,0.0,0.0,0.0,0.0,4580027352,000ms_ho,1681.0,634422,みりほーちゃんがフォロバしたユーザーが、よるほーや時報をピッタリに呟くと数秒後にピタオメリプ...,みりほーのトリセツ：詳しくは、Web見てね♪ 固定ツイートも見てね♪,5339,533946,01,03
2,163,1669,時々女子大生です☆〜（ゝ。∂）,上野動物園キャンパス,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5339,533946,,
3,166,1679,208,こちら03:59。鳥のさえずりが聞こえ始めてます。,2022-07-01_03:59:57,0.0,0.0,2.0,0.0,1416380292512706560,NARIPON_T,192.0,4064,💢VALORANT┊最高A3 現A2🔫APEX┊( ᐙ )🕹ガンエボ┊シルバー2,NaN,5339,533946,01,03
4,220,2361,391,たこ焼きとからあげクンな晩ご飯えびが好き♡ひじきとワカメ入りサラダ。#晩御飯 ＃晩ごはん #...,2022-07-01_04:59:54,3.0,0.0,34.0,0.0,2491409160,7717Hi,265.0,2349,好きなものを自由に。好きなことが一緒の方のフォロー大歓迎です。料理、お酒、セカオワ、vaun...,NaN,5339,533946,01,04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,30969,349991,152,おはようございます。お返事ありがとうございます。縦の三玖ちゃんを求めておりましたので今回は...,2022-07-31_08:59:58,0.0,0.0,1.0,0.0,942390297715023872,ldh_36829,146.0,772,交換垢 成人済み,NaN,5339,533946,31,08
639,31094,351502,203,RT : 秋葉原駅内イベント開催中です！ HOW HOUSE PICK UP CREATOR...,2022-07-31_11:59:58,6.0,0.0,0.0,0.0,4310858292,chii_nekomatagi,338.0,112826,雑食系。面白そうなモノはまず齧ってみる。典型「熱しやすく冷めやすい」ASD系の障害手帳持ち。...,南関東のブラックホール,5339,533946,31,11
640,31144,352108,321,久しぶりに青春18で東京いくのもありか・・・,2022-07-31_12:59:58,0.0,0.0,3.0,0.0,1507245180138651650,0108siHiroto,229.0,6328,京都工芸繊維大 情報工学 B1 | KMC,京都,5339,533946,31,12
641,31219,352828,87,RT : やった！あかりちゃんの絵柄が入ったうちわもらいました。あとブロマイドも当選。しかも...,2022-07-31_14:59:58,1.0,0.0,0.0,0.0,1477765710663516160,yryryrmgr,40.0,6156,#ゆるゆり活動日記 をひたすら巡回するだけの非公式アカウントです。ゆる〜くやってます。,NaN,5339,533946,31,14
